# Transformer Anatomy

The Transformer architecture, introduced in the paper "Attention is All You Need" by Vaswani et al., 2017, has revolutionized the field of NLP. Unlike previous models that relied on recurrent or convolutional layers, Transformers use self-attention mechanisms to capture dependencies between words in a sentence, regardless of their distance.

### Key Components of the Transformer Architecture:

1. **Positional Encoding**: Adds information about the position of words in a sequence since the model itself does not inherently understand word order.
2. **Self-Attention Mechanism**: Allows the model to weigh the importance of each word in a sentence relative to all other words.
3. **Multi-Head Attention**: Enables the model to focus on different parts of the input simultaneously.
4. **Layer Normalization and Residual Connections**: Stabilizes training and helps with gradient flow.
5. **Feed-Forward Neural Networks**: Applies a point-wise feed-forward layer to each position independently and destilles the information further to output probabilities.

Let's explore each of these components in detail and understand how they work together to create powerful language models.

## Self-Attention Mechanism

The self-attention mechanism is the core component of the Transformer architecture. It allows the model to dynamically assign different levels of importance to different words in a sentence when encoding a particular word.

### How Self-Attention Works:

1. **Input Embeddings**: Before we can apply self-attention, the input words must first be converted into embeddings (dense vector representations).
2. **Query, Key, and Value Vectors**: For each word, the model creates three vectors: a Query vector (Q), a Key vector (K), and a Value vector (V).
3. **Attention Scores**: The attention score is computed as the dot product of the Query vector of a word with the Key vectors of all words. This score determines how much focus should be on the other words.
4. **Weighted Sum**: Each word's output representation is computed as a weighted sum of the Value vectors, where the weights are the normalized attention scores.
5. **Softmax Normalization**: After the attention heads, the scores are passed through a softmax function to convert them into probabilities.

### Visualizing Self-Attention

Let's visualize how the self-attention mechanism works for a simple sentence.


In [ ]:
# Import required libraries
import torch
import torch.nn.functional as F
import math

In [ ]:
# Example sentence and tokens
sentence = "Transformers are revolutionary in NLP."
tokens = ["Transformers", "are", "revolutionary", "in", "NLP"]

In [ ]:
# Embedding dimension
embedding_dim = 8

# Random input embeddings (for illustration purposes)
torch.manual_seed(42)
input_embeddings = torch.randn(len(tokens), embedding_dim)

In [ ]:
# In this example we end up with a 5x8 matrix
input_embeddings

In [ ]:
# Initialize Query, Key, and Value weight matrices
Q = torch.randn(embedding_dim, embedding_dim)
K = torch.randn(embedding_dim, embedding_dim)
V = torch.randn(embedding_dim, embedding_dim)

In [ ]:
# Compute Query, Key, and Value vectors
queries = input_embeddings @ Q
keys = input_embeddings @ K
values = input_embeddings @ V

In [ ]:
# Calculate attention scores using dot product of queries and keys
attention_scores = queries @ keys.T

# Apply softmax to normalize the scores
attention_weights = F.softmax(attention_scores/math.sqrt(embedding_dim), dim=-1)

In [ ]:
# Compute the weighted sum of values
output = attention_weights @ values

# Display the attention weights
print("Attention Weights:\n", attention_weights)

In [ ]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

In [ ]:
model_dir = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/google--bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = BertModel.from_pretrained(model_dir)

In [ ]:
text = "The woman at the bus stop looked really cheerful."

In [ ]:
show(model, "bert", tokenizer, text, display_mode="light", layer=11, head=8)

## Multi-Head Attention

The multi-head attention mechanism allows the model to focus on different parts of the input simultaneously. Instead of having a single attention mechanism, the model uses multiple attention "heads" in parallel. Each head can learn different aspects of the input.

### How Multi-Head Attention Works:

1. The input is projected into multiple sets of Query, Key, and Value vectors.
2. Each set of vectors is processed independently through a self-attention mechanism.
3. The outputs from each head are concatenated and projected back into a single vector space.

This approach provides the model with a richer understanding of the input by capturing different types of relationships between words.

### Example: Multi-Head Attention
Let's visualize how multi-head attention works using multiple attention heads.


In [ ]:
# Number of attention heads
num_heads = 2

# Initialize weight matrices for each head
Q_heads = [torch.randn(embedding_dim, embedding_dim) for _ in range(num_heads)]
K_heads = [torch.randn(embedding_dim, embedding_dim) for _ in range(num_heads)]
V_heads = [torch.randn(embedding_dim, embedding_dim) for _ in range(num_heads)]

In [ ]:
# Compute outputs for each head
head_outputs = []
for i in range(num_heads):
    queries = input_embeddings @ Q_heads[i]
    keys = input_embeddings @ K_heads[i]
    values = input_embeddings @ V_heads[i]
    
    # Calculate attention scores and apply softmax
    attention_scores = queries @ keys.T
    attention_weights = F.softmax(attention_scores/math.sqrt(embedding_dim), dim=-1)
    
    # Compute the weighted sum of values
    output = attention_weights @ values
    head_outputs.append(output)

In [ ]:
# Concatenate outputs from all heads
multi_head_output = torch.cat(head_outputs, dim=-1)

# Display multi-head attention output
print("Multi-Head Attention Output:\n", multi_head_output)

In [ ]:
from bertviz import head_view
from transformers import AutoModel

In [ ]:
model = AutoModel.from_pretrained(model_dir, output_attentions=True)

In [ ]:
sentence_1 = "time flies like an arrow"
sentence_2 = "fruit flies like a banana"

In [ ]:
viz_inputs = tokenizer(sentence_1, sentence_2, return_tensors="pt")
attention = model(**viz_inputs).attentions
sentence_2_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

In [ ]:
head_view(attention, tokens, sentence_2_start, heads=[8])

## Feed-Forward Neural Networks

Each position's output from the multi-head attention mechanism is passed through a point-wise feed-forward neural network. This consists of two linear transformations with a ReLU activation in between.

### Example: Feed-Forward Network
Let's implement a simple feed-forward network.

In [ ]:
# Define feed-forward neural network
class FeedForwardNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForwardNN, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        return self.linear2(self.relu(self.linear1(x)))

In [ ]:
# Instantiate and apply the feed-forward network
ffn = FeedForwardNN(input_dim=embedding_dim * num_heads, hidden_dim=32)
ffn_output = ffn(multi_head_output)

# Display the feed-forward network output
print("Feed-Forward Network Output:\n", ffn_output)

## Layer Normalization and Residual Connections

Layer normalization is used to stabilize training by normalizing the inputs to each layer. Residual connections help maintain gradient flow through the network, enabling deeper architectures.

### Example: Adding Layer Normalization and Residual Connections
Let's see how these components are added to the Transformer block.


In [ ]:
# Define Layer Normalization
layer_norm = torch.nn.LayerNorm(embedding_dim * num_heads)

# Add residual connection and apply layer normalization
residual_output = layer_norm(multi_head_output + ffn_output)

# Display the final output with residual connection
print("Output with Residual Connections:\n", residual_output)

## Positional Encoding

Since the Transformer does not inherently capture the order of words, positional encoding is added to provide the model with information about the relative position of words in a sentence.

### Example: Implementing Positional Encoding
Let's implement positional encoding for a sequence of words.


In [ ]:
import numpy as np

def positional_encoding(seq_len, model_dim):
    pos_enc = np.zeros((seq_len, model_dim))
    for pos in range(seq_len):
        for i in range(0, model_dim, 2):
            pos_enc[pos, i] = np.sin(pos / (10000 ** (2 * i / model_dim)))
            pos_enc[pos, i + 1] = np.cos(pos / (10000 ** (2 * i / model_dim)))
    return torch.tensor(pos_enc, dtype=torch.float)

In [ ]:
# Random input embeddings (for illustration purposes)
torch.manual_seed(42)
input_embeddings = torch.randn(len(tokens), embedding_dim)

In [ ]:
# Apply positional encoding
position_encodings = positional_encoding(len(tokens), embedding_dim)

# Add positional encoding to input embeddings
encoded_input = input_embeddings + position_encodings

print("Positional Encodings:\n", position_encodings)
print("Encoded Input with Positional Information:\n", encoded_input)

## Conclusion

In this notebook, we explored the key components of the Transformer architecture, including self-attention, multi-head attention, feed-forward networks, layer normalization, and positional encoding. These components work together to form the basis of modern NLP models.

In [ ]:
# Shut down the kernel to release memory
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)